## EthSF 24 Test Notebook

# A. Owner

### 1. Create a new session as owner

In [ ]:
from weaveapi.records import *
from weaveapi.options import *
from weaveapi.filter import *
from weaveapi.weaveh import *

import pandas as pd

from web3.auto import w3
from eth_account.messages import encode_defunct
from hexbytes import HexBytes

WEAVE_CONFIG = "config/demo_client_ethan.config"
nodeApiOwner, sessionOwner = connect_weave_api(WEAVE_CONFIG)

organization = "ethan"

data_collection = "vault"
files_table = "files"
wallets_table = "wallets"
hashes_table = "hashes"

### 2. Create a table to store files

In [ ]:
layout = { 
    "columns": { 
        "id": { "type": "LONG", "isIndexed": True, "isUnique": True, "isNullable": False },
        "ts": { "type": "LONG" },
        "writer": { "type": "STRING" },
        "sig": { "type": "STRING" },
        "access": { "type": "STRING" },
        "did": { "type": "STRING" },
        "name": { "type": "STRING" },
        "recipient": { "type": "STRING" },
        "content": { "type": "STRING" }
    }, 
    "idColumnIndex": 0,  # Autogenerates IDs
    "timestampColumnIndex": 1, # Fills the column automatically with the network time
    "ownerColumnIndex": 2, # Fills the pubkey column automatically with the public key of the writer
    "signatureColumnIndex": 3, # Fills the column with an EdDSA signature of the record hash
    "allowedRolesColumnIndex": 4, # Fills the column with the allowed reader
    "accessExpiryColumnIndex": 7, # Fills the column with the allowed reader
    "isLocal": False,
    "applyReadTransformations": True
}

nodeApiOwner.dropTable(sessionOwner, data_collection, files_table).get()
reply = nodeApiOwner.createTable(sessionOwner, data_collection, files_table, CreateOptions(False, False, layout)).get()
print(reply)